In [1]:
# necessary imports
import json
import os
import torch.nn.functional as F

In [ ]:
def set1(v_name):
    action, number = v_name.split('_')
    labels = {
        'handShake': {2,14,15,16,18,19,20,21,24,25,26,27,28,32,40,41,42,43,44,45,46,47,48,49,50},
	    'highFive' : {1,6,7,8,9,10,11,12,13,23,24,25,27,28,29,30,31,32,33,34,35,44,45,47,48},
	    'hug'      : {2,3,4,11,12,15,16,17,18,20,21,27,29,30,31,32,33,34,35,36,42,44,46,49,50},
	    'kiss'	   : {1,7,8,9,10,11,12,13,14,16,17,18,22,23,24,26,29,31,35,36,38,39,40,41,42},
        'negative' : {i for i in range(1, 51)}
    }
    return (int(number) in labels[action])

In [ ]:
classes = {
    'negative' : 0, 
    'handShake': 1,
    'highFive' : 2,
    'hug'      : 3,
    'kiss'     : 4}

In [ ]:
def load_files(v_name, data_video_path):
    for videos in sorted(os.listdir(os.path.join(data_video_path, v_name))):
        if len(videos.split('_')) != 1:
            continue
        for i in json.load(open(os.path.join(data_video_path, v_name, videos))):
            json.dump([i], open(os.path.join(data_video_path, v_name, i['video']), 'w'))
        ! rm $data_video_path/$v_name/$videos


In [2]:
def create_dataset(data_video_path, data_audio_path):

    datasets = [[], []]

    for video in sorted(os.listdir(data_video_path)):
    
        audio_loc = os.path.join(data_audio_path, f'{video}.json')

        if video[0] == '.': continue
        #if '' != data_audio_path and video not in os.listdir(data_audio_path): continue

        #    continue
        set_no = int(not set1(video))

        for track_name in sorted(os.listdir(os.path.join(data_video_path, video))):

            if track_name[0] == '.': continue

            track = track_name.split('.')[0]
            extension = '.'.join(track_name.split('.')[1:])
            lead, track1, track2, v, no_frames, interac_class = track.split('_')[:6]
            if int(no_frames) < 2: continue

            if 'aug2' in track: aug = 2
            elif 'aug1' in track: aug = 1
            else: aug = 0

            if int(track1) > int(track2):
                continue

            to_join = [lead, track2, track1, v, no_frames, interac_class]            
            if aug: to_join.append(f'aug{aug}')
            pair_name = f"{'_'.join(to_join)}.{extension}"

            file_loc = os.path.join(data_video_path, video, track_name)
            pair_loc = os.path.join(data_video_path, video, pair_name)
            audio_name = f"{'_'.join([lead, track1, track2, v, no_frames, interac_class])}.mp4"
            audio_loc = os.path.join(data_audio_path, video, audio_name)

            if os.path.getsize(file_loc) == 0:  continue
            if (not os.path.isfile(pair_loc)) or os.path.getsize(pair_loc) == 0:  continue
            if not os.path.isfile(audio_loc): continue

            feature1 = json.load(open(file_loc, 'rb'))
            feature2 = json.load(open(pair_loc, 'rb'))

            feature_audio = json.load(open(audio_loc))


            if feature1 == [] or feature2 == []:
                continue
            assert len(feature1) == 1 and len(feature2) == 1

            video_name = feature1[0]['video']
            if video_name != f'{track}.mp4':
                continue

            pair_name = f"{'_'.join([lead, track2, track1, v, no_frames, interac_class])}.mp4"
            
            for (seg1, seg2) in zip(feature1[0]['clips'], feature2[0]['clips']):
                datasets[set_no].append((seg1['features'],
                                         seg2['features'],
                                         feature_audio,
                                         int(interac_class)))

            assert len(feature1[0]['clips']) == len(feature2[0]['clips']) 
            
    return datasets


In [3]:
def create_dataset(episode_no, data_video_path, data_audio_path, type='train'):
    
    res = []
    episode_str = f'output_episode{str(episode_no).zfill(2)}'
    augmentations = ['', '_aug1', '_aug2'] if type == 'train' else ['']
    if episode_no == 18:
        augmentations = ['', '_aug1'] if type == 'train' else ['']

    for aug in augmentations:
        for (label, mode) in enumerate(['_nointeracs', '']):

            path = f'{data_video_path}/{episode_str + mode + aug}'
    
            for v_feature in sorted(os.listdir(path)):
                
                curr_file = f'{path}/{v_feature}'
                track1, track2, v = v_feature[5:].split('.')[0].split('_')

                if int(track1) > int(track2):
                    continue

                extension = '.'.join(v_feature.split('.')[1:])
                pair_loc = f'video{str(track2).zfill(3)}_{str(track1).zfill(3)}_{v}.{extension}'
                pair_file = f'{path}/{pair_loc}'
                audio_loc = f'audio_{str(track1).zfill(3)}_{str(track2).zfill(3)}_{v}_{label}.json'
                audio_file = f'{data_audio_path}/episode{str(episode_no).zfill(2)}/{audio_loc}'

                if os.path.getsize(curr_file) == 0:  continue
                if (not os.path.isfile(pair_file)) or os.path.getsize(pair_file) == 0:
                    continue
                
                feature1 = json.load(open(curr_file, 'rb'))
                feature2 = json.load(open(pair_file, 'rb'))
                feature_audio = json.load(open(audio_file)) if os.path.isfile(audio_file) else []

                if feature1 == [] or feature2 == []: continue
                assert len(feature1) == 1 and len(feature2) == 1

                if len(feature1[0]['clips']) != len(feature2[0]['clips']):
                    continue
                
                for (seg1, seg2) in zip(feature1[0]['clips'], feature2[0]['clips']):
                    res.append((seg1['features'],
                                seg2['features'],
                                feature_audio,
                                int(label)))
                                
    return res 

    

In [13]:

data_video_path = f'../classification/features/output'
data_audio_path = f'../data/audio_features'

datasets = create_dataset(data_video_path, data_audio_path)
for i in range(2):
    json.dump(datasets[i], open(f'../data/video_dataset/features_set{i + 1}_valid.json', 'w'))
    print(len(datasets[i]))

datasets_aug1 = create_dataset(data_video_path + '_aug1', data_audio_path)
datasets_aug2 = create_dataset(data_video_path + '_aug2', data_audio_path)

for i in range(2):
    json.dump(datasets[i] + datasets_aug1[i] + datasets_aug2[i],
                open(f'../data/video_dataset/features_set{i + 1}_train.json', 'w'))
    print(len(datasets_aug1[i]), len(datasets_aug2[i]))